In [1]:
import os
os.chdir ("/Users/neba/Desktop/CyclicaVsFDA/TEST_cyclica_FDA copy")
os.getcwd()

!pwd

/Users/neba/Desktop/CyclicaVsFDA/TEST_cyclica_FDA copy


In [2]:
!ls -l
!pwd

total 104160
-rw-r--r--   1 neba  staff  6931030  3 Jun 03:56 Untitled1-Copy1.ipynb
-rw-r--r--   1 neba  staff    61782  3 Jun 04:46 Untitled1-Copy2.ipynb
-rw-r--r--   1 neba  staff  6923827  3 Jun 04:14 Untitled1-Copy3.ipynb
-rw-r--r--   1 neba  staff  6923831  3 Jun 03:38 Untitled1-Copy4.ipynb
-rw-r--r--   1 neba  staff  6923831  3 Jun 04:16 Untitled1-Copy5.ipynb
-rw-r--r--   1 neba  staff    62889  3 Jun 04:50 Untitled1-Copy6.ipynb
-rw-r--r--   1 neba  staff    60939  6 Jun 12:37 Untitled1.ipynb
-rw-r--r--   1 neba  staff     4231 30 May 21:57 control_cyc1.pdb
-rw-r--r--   1 neba  staff     6118 30 May 21:58 control_cyc2.pdb
-rw-r--r--   1 neba  staff      636  2 Jun 18:01 control_fda1.pdb
-rw-r--r--   1 neba  staff     4231 30 May 21:57 cyclica1.pdb
-rw-r--r--   1 neba  staff     6118 30 May 21:58 cyclica2.pdb
-rw-r--r--   1 neba  staff     2611  2 Jun 17:56 fda1000.pdb
-rw-r--r--   1 neba  staff     1663  2 Jun 17:56 fda901.pdb
-rw-r--r--   1 neba  staff     2058  2 Jun 17:56 fda9

/Users/neba/Desktop/CyclicaVsFDA/TEST_cyclica_FDA copy


In [ ]:
#!/usr/bin/python
#
# PairWise_RMSD_smalLigs.py
 
import __main__
__main__.pymol_argv = [ 'pymol', '-qc'] # Quiet and no GUI
 
import sys, time, os, re
from pymol import cmd
import pymol
import glob
import pandas as pd

pymol.finish_launching()

directory = "/Users/neba/Desktop/CyclicaVsFDA/TEST_cyclica_FDA copy/*.pdb"
#directory2 = "/Users/neba/Desktop/CyclicaVsFDA/Results/control*.pdb"


# Read User Input
structurePath = glob.glob(directory)
#structurePath2 = glob.glob(directory2)

structurePath.sort()
#structurePath2.sort()
extension = re.compile( '(^.*[\/]|\.(pdb|ent|brk))' )
selection=None
    
rmsd = {}    
rmsd_list = []
for i in range (len(structurePath)):
    for j in range(1+i, len(structurePath)):
        obj_name1 = extension.sub('',structurePath[i].split("/")[-1].split(".")[0])
        cmd.load(structurePath[i],obj_name1)
        obj_name2 = extension.sub('',structurePath[j].split("/")[-1].split(".")[0])
        cmd.load(structurePath[j],obj_name2)
        
        rmsd_val = cmd.align(" %s , %s" %(obj_name2, selection),
                             " %s , %s" %(obj_name1, selection),
                             cutoff=200,cycles=200)
        
        rmsd.setdefault(obj_name1,{})[obj_name2] = rmsd_val[0],rmsd_val[3],
        rmsd_val[4],rmsd_val[5]
        
        rmsd_list.append((rmsd_val[3], rmsd_val[0], rmsd_val[4],
                          rmsd_val[5], obj_name1, obj_name2)) #indexing to creat
                                                                          #columns in df
        
        #Delete object and replace with new items when 
        #loop loops through directory, the delete ccommnds must
        #align with loops for good results
        cmd.delete(obj_name2)
    cmd.delete(obj_name1)

#Re-order table
rmsd_list.sort()

#testing
print (rmsd_list)

#Create tables with poutput pairwise rmsds
rmsd_df = pd.DataFrame(rmsd_list)

#Renaming columns to match pymol's "align" column names
rmsd_df.columns = ['RMSD_BR','RMSD_AR', '#atomsAligned',
                   'Alignment_score','Ref_Ligand', 
                   'Aligned_Ligand']

#save file to directory
rmsd_df.to_csv("rmsd700-900.csv", index=False)

rmsd_df.head(10)

#The "align" tool of pymol returns a list with 7 items:
#RMSD after refinement
#Number of aligned atoms after refinement
#Number of refinement cycles
#RMSD before refinement
#Number of aligned atoms before refinement
#Raw alignment score
#Number of residues aligned
#


Cannot find license file.
 The license files (or license server system network addresses) attempted are 
listed below.  Use LM_LICENSE_FILE to use a different license file,
 or contact your software provider for a license file.
Feature:       PYMOL_MAIN
Filename:      /Library/Application Support/Schrodinger/licenses
License path:  /Library/Application Support/Schrodinger/licenses:
FlexNet Licensing error:-1,359.  System Error: 2 "No such file or directory"


 Matrix: Warning: no convergence (0.00000010<189.23849487 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<5.98077106 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<189.23858643 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<73.39413452 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<92.90372467 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<92.04867554 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<48.43562317 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<370.78292847 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<23.61342621 after 1000 iterations).


In [ ]:
pd.read_csv('rmsd700-900.csv')

In [ ]:
rmsd_df[(rmsd_df["RMSD_BR"] < 2) & (rmsd_df["Alignment_score"] > 3)
        & (rmsd_df['#atomsAligned'] > 5)].head(50)